# AUV 3D

In [1]:
import sys
sys.path.append("../src/dynamics")
from pontryagin import system
from sympy import init_printing
from sympy import *
init_printing()

In [2]:
# state
x, y, z, vx, vy, vz, qr, qx, qy, qz = symbols('x y z v_x v_y v_z q_r q_x q_y q_z')
p                      = Matrix([x, y, z])
v                      = Matrix([vx, vy, vz])
q                      = Matrix([qr, qx, qy, qz])
s                      = Matrix([p, v, q])
s.T

[x  y  z  vₓ  v_y  v_z  qᵣ  qₓ  q_y  q_z]

In [3]:
# controls: throttle, reaction wheels
ut, ux, uy = symbols("u_t u_x u_y")
u = Matrix([ut, ux, uy])
u.T

[uₜ  uₓ  u_y]

In [4]:
# physical parametres
mass, density, CD, planaform, thrust, rotation = symbols("m rho cd A T omega")

# fluid velocity
vfx, vfy, vfz = symbols("v_xf v_yf v_zf")
vf = Matrix([vfx, vfy, vfz])

# environmental parametres
params = [*vf, mass, density, CD, planaform, thrust, rotation]

# optimisation parametres
alpha = symbols("alpha")

In [5]:
# free stream velocity
vinf = v - vf

# free stream velocity magnitude
vinfmag = sqrt(sum([var**2 for var in vinf]))

# fluid drag
Fd = - Rational(1,2)*density*vinf*vinfmag*CD*planaform

# thrust direction
ix = 2*(qx*qz - qy*qr)
iy = 2*(qy*qz - qx*qr)
iz = 1 - 2*(qx**2 + qy**2)
i  = Matrix([ix, iy, iz])

# thrust
Ft = thrust*ut*i

In [6]:
# translation
dp = v
dv = (Fd + Ft)/mass

# rotation
dq = rotation*Rational(1,2) * Matrix([
    [0, 0, -uy, ux],
    [0, 0, ux, uy],
    [uy, -ux, 0, 0],
    [-ux, -uy, 0, 0]
]) * q


# state transition
ds = Matrix([dp, dv, dq])
ds

⎡                                                 vₓ                          
⎢                                                                             
⎢                                                 v_y                         
⎢                                                                             
⎢                                                 v_z                         
⎢                                                                             
⎢                        ______________________________________________       
⎢                       ╱            2               2               2        
⎢  A⋅cd⋅ρ⋅(vₓ - v_xf)⋅╲╱  (vₓ - v_xf)  + (v_y - v_yf)  + (v_z - v_zf)         
⎢- ──────────────────────────────────────────────────────────────────── + T⋅uₜ
⎢                                   2                                         
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                 m 

In [7]:
# Lagrangian
L = (1-alpha)*(ut**2 + ux**2 + uy**2) + alpha*(ut + ux**2 + uy**2)
L

  ⎛       2      2⎞            ⎛  2     2      2⎞
α⋅⎝uₜ + uₓ  + u_y ⎠ + (-α + 1)⋅⎝uₜ  + uₓ  + u_y ⎠

In [8]:
# equality constraints
iq = [ut-1, -ut, ux-1, -ux-1, uy-1, -uy-1]

In [9]:
# initialise system
sys = system(s, u, ds, L, inequality=iq)

In [10]:
sys.solve()

⎡⎧                                                                            
⎢⎪                                              -T⋅λᵥ ₓ⋅qᵣ⋅q_y + T⋅λᵥ ₓ⋅qₓ⋅q_z
⎢⎨                                                                            
⎢⎪η₀: 0, η₁: 0, η₂: 0, η₃: 0, η₄: 0, η₅: 0, uₜ: ──────────────────────────────
⎣⎩                                                                            

                                               2              2   T⋅λ_v_z   α⋅
 - T⋅λ_v_y⋅qᵣ⋅qₓ + T⋅λ_v_y⋅q_y⋅q_z - T⋅λ_v_z⋅qₓ  - T⋅λ_v_z⋅q_y  + ─────── + ──
                                                                     2       2
──────────────────────────────────────────────────────────────────────────────
                    m⋅(α - 1)                                                 

m                                                                             
─                                                                             
       ω⋅(-λ_q_r⋅q_z - λ_q_x⋅q_y + λ_q_y⋅qₓ + λ_q_